In [1]:
## Can also get the price chart by pokemon number, or url for past prices. Look intro trading strategies; also, work on real-time offering feed vs history. 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re

# Customize number of pages to scrape
num_pages = 1
base_url = 'https://www.tcgplayer.com/search/pokemon/product?productLineName=pokemon&view=grid&Condition=Near+Mint|Lightly+Played&page={}&Language=English'

# Setup headless Chrome options
options = Options()
options.add_argument("--headless=new")
prefs = {"profile.default_content_setting_values": {"images": 2, "stylesheets": 2}}
options.add_experimental_option("prefs", prefs)

def scrape_page(page_num):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    url = base_url.format(page_num)
    driver.get(url)

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card__product")))
    except:
        driver.quit()
        return []

    products = driver.find_elements(By.CLASS_NAME, "product-card__product")
    results = []

    for product in products:
        try:
            # Grab the link directly from the <a> tag
            link_elem = product.find_element(By.XPATH, "./ancestor::a")
            product_link = link_elem.get_attribute("href")
        except Exception:
            product_link = None

        try:
            img_elem = product.find_element(By.TAG_NAME, "img")
            name = img_elem.get_attribute("alt").strip()
        except Exception:
            name = None

        try:
            set_name = product.find_element(By.CLASS_NAME, "product-card__set-name__variant").text.strip()
        except Exception:
            set_name = None

        try:
            mktprice_text = product.find_element(By.CLASS_NAME, "product-card__market-price--value").text.strip()
            mktprice_match = re.search(r"\$([\d,.]+)", mktprice_text)
            mktprice = float(mktprice_match.group(1).replace(",", "")) if mktprice_match else None
        except Exception:
            mktprice = None

        try:
            listings_span = product.find_element(By.CLASS_NAME, "inventory__listing-count").text.strip()
            listings_match = re.search(r"(\d+)\s+listings", listings_span)
            listings = int(listings_match.group(1)) if listings_match else None
        except Exception:
            listings = None

        results.append({
            "name": name,
            "link": product_link,
            "set": set_name,
            "mktprice": mktprice,
            "listings": listings
        })

    driver.quit()
    return results

# Run across multiple pages
all_results = []
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(scrape_page, p) for p in range(1, num_pages + 1)]
    for future in as_completed(futures):
        all_results.extend(future.result())

# Convert to DataFrame
pricedf = pd.DataFrame(all_results)
print(pricedf)


                                             name  \
0        Code Card - Destined Rivals Booster Pack   
1   Code Card - Prismatic Evolutions Booster Pack   
2            Code Card - White Flare Booster Pack   
3             Code Card - Black Bolt Booster Pack   
4                                           Hilda   
5                                     Air Balloon   
6                                 Arven - 166/198   
7                                         Pikachu   
8                                   Victini - 208   
9                                  Iono - 080/091   
10                                   Prism Energy   
11                                Night Stretcher   
12                                   Brave Bangle   
13                                Luminous Energy   
14                        Black Bolt Booster Pack   
15                             Buddy-Buddy Poffin   
16                                         Kyogre   
17                       White Flare Booster P

In [3]:
pricedf

,name,link,set,mktprice,listings
0,Code Card - Destined Rivals Booster Pack,https://www.tcgplayer.com/product/633169/pokem...,SV10: Destined Rivals,0.05,222.0
1,Code Card - Prismatic Evolutions Booster Pack,https://www.tcgplayer.com/product/614046/pokem...,SV: Prismatic Evolutions,0.04,222.0
2,Code Card - White Flare Booster Pack,https://www.tcgplayer.com/product/646130/pokem...,SV: White Flare,0.08,95.0
3,Code Card - Black Bolt Booster Pack,https://www.tcgplayer.com/product/646128/pokem...,SV: Black Bolt,0.05,98.0
4,Hilda,https://www.tcgplayer.com/product/642200/pokem...,SV: White Flare,3.74,163.0
5,Air Balloon,https://www.tcgplayer.com/product/642531/pokem...,SV: Black Bolt,1.57,261.0
6,Arven - 166/198,https://www.tcgplayer.com/product/488071/pokem...,SV01: Scarlet & Violet Base Set,2.46,546.0
7,Pikachu,https://www.tcgplayer.com/product/250303/pokem...,Celebrations,7.23,257.0
8,Victini - 208,https://www.tcgplayer.com/product/646169/pokem...,SV: Scarlet & Violet Promo Cards,7.15,183.0
9,Iono - 080/091,https://www.tcgplayer.com/product/534442/pokem...,SV: Paldean Fates,0.29,913.0


Login failed or not required: Message: 
Stacktrace:
0   chromedriver                        0x000000010bebeef8 chromedriver + 5918456
1   chromedriver                        0x000000010beb648a chromedriver + 5883018
2   chromedriver                        0x000000010b982e20 chromedriver + 429600
3   chromedriver                        0x000000010b9d4ed4 chromedriver + 765652
4   chromedriver                        0x000000010b9d50f1 chromedriver + 766193
5   chromedriver                        0x000000010ba24ce4 chromedriver + 1092836
6   chromedriver                        0x000000010b9faf3d chromedriver + 921405
7   chromedriver                        0x000000010ba22024 chromedriver + 1081380
8   chromedriver                        0x000000010b9face3 chromedriver + 920803
9   chromedriver                        0x000000010b9c729b chromedriver + 709275
10  chromedriver                        0x000000010b9c7f81 chromedriver + 712577
11  chromedriver                        0x000000010be

OSError: [Errno 30] Read-only file system: 'scraped_sales_data_parallel.csv'

In [ ]:
print(salesdf)


{'https://www.tcgplayer.com/product/633169/pokemon-sv10-destined-rivals-code-card-destined-rivals-booster-pack?Condition=Near+Mint|Lightly+Played&Language=English&page=1':       date condition  qty  price
0  8/16/25        NM    6  $0.04
1  8/16/25        NM    7  $0.04
2  8/16/25        NM    2  $0.03
3  8/16/25        NM  100  $0.06
4  8/15/25        NM   99  $0.05}


In [ ]:
/Users/nshaffer/Library/Application Support/Google/Chrome/Profile 5